In [2]:
import numpy as np
import pandas as pd
df_2015 = pd.read_csv('./data/wh/2015.csv')
df_2016 = pd.read_csv('./data/wh/2016.csv')
df_2017 = pd.read_csv('./data/wh/2017.csv')

years_dict = {2015: df_2015, 2016: df_2016, 2017: df_2017}

# standarize column names along all years
for y, df in years_dict.items():
    if y > 2015:
        df = df.rename(columns={
            'Trust..Government.Corruption': 'Trust (Government Corruption)', 
            'Economy..GDP.per.Capita.': 'Economy (GDP per Capita)',
            'Health..Life.Expectancy.': 'Health (Life Expectancy)'
        })
        df.columns = df.columns.str.replace('.', ' ')
        years_dict[y] = df

# normalize happiness score in terms of min and max 
for y, df in years_dict.items():
    max_score = df['Happiness Score'] .max()                                                                                 
    min_score = df['Happiness Score'] .min()
    df['Happiness Score Normalized'] = df['Happiness Score'] .apply(lambda x: (x - min_score) / (max_score - min_score))

# choropleth map
import plotly.graph_objs as go

from IPython.display import HTML
from plotly import __version__
from plotly.offline import download_plotlyjs, init_notebook_mode, plot, iplot
init_notebook_mode(connected=True)

csc = scl = [
    [0.0, 'rgb(242,240,247)'],
    [0.2, 'rgb(218,218,235)'],
    [0.4, 'rgb(188,189,220)'],
    [0.6, 'rgb(158,154,200)'],
    [0.8, 'rgb(117,107,177)'],
    [1.0, 'rgb(84,39,143)']
]

# build slider over the three years
years_map = []
for y, df in years_dict.items():
    data = [dict(
        type='choropleth',
        locations = df['Country'],
        locationmode = 'country names', 
        z = df['Happiness Score'].astype(float),
        text = df['Country'], 
        autocolorscale = False,
        colorbar = {'title':'Happiness Rank'},
        colorscale= csc,    
        reversescale = False
    )]
    years_map.extend(data)
   

# build steps
years = list(years_dict.keys())
steps = []
total_years = len(years_map)
for i in range(0, total_years):
    step = dict(method = "restyle",
                args = ["visible", [False] * total_years],
                label = years[i]) 
    step['args'][1][i] = True
    steps.append(step)
    
# build slider
sliders = [
    dict(
        active = 10,
        currentvalue = {"prefix": "Year: "},
        pad = {"t": 50},
        steps = steps
    )
]

layout = dict(title='World Happiness Rank', sliders=sliders)
fig = dict(data=years_map, layout=layout)
iplot( fig, filename='World-Happiness-Rank' )

In [3]:
from collections import defaultdict

# compare first and last countries
first_countries = df_2015['Country'].iloc[0:5]
last_countries = df_2015['Country'].iloc[-5:]

first_scores = defaultdict(list)
last_scores = defaultdict(list)

# build dictionary showing change over years
for country in first_countries:
    for y, df in years_dict.items():
        country_data = df.loc[df['Country'] == country]
        first_scores[country].append(country_data['Happiness Score'].values[0])
        
for country in last_countries:
    for y, df in years_dict.items():
        country_data = df.loc[df['Country'] == country]
        last_scores[country].append(country_data['Happiness Score'].values[0])
        

# build line charts
line_charts = []
for country in first_countries:
    data = dict(
        type='scatter',
        name=country,
        x=years,
        y=first_scores[country]
    )
    line_charts.append(data)
    
layout = dict(
    title = 'Change over years First 5 countries',
    xaxis = dict(title = 'Year'),
    yaxis = dict(title = 'Happiness Score'),
)
iplot( line_charts, layout, filename='Change over years' )

line_charts = []
for country in last_countries:
    data = dict(
        type='scatter',
        name=country,
        x=years,
        y=last_scores[country]
    )
    line_charts.append(data)
    
layout = dict(
    title = 'Change over years Last 5 countries',
    xaxis = dict(title = 'Year'),
    yaxis = dict(title = 'Happiness Score'),
)
iplot( line_charts, layout, filename='Change over years' )